In [ ]:
from bindingcalculator import BindingCalculator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pickle

## Load binding calculator

In [ ]:
# instantiate binding calculator based on escape data
# https://github.com/jbloomlab/SARS2_RBD_Ab_escape_maps/

metric='sum of mutations at site' # metric='mean of mutations at site',
bindcalc = BindingCalculator(csv_or_url='./escape_calculator_data.csv',
                             eliciting_virus='SARS-CoV-2',
                             source_lab='all',
                             neutralizes_Omicron='either',
                             metric=metric,
                             mutation_escape_strength=2.0)
sites = bindcalc.sites
print("A total of {} RBD sites have escape data".format(len(bindcalc.sites)))
print("Sites span {} to {}".format(min(sites), max(sites)))

In [ ]:
# ingest pyro-cov inferences
mutations = pd.read_csv('../paper/mutations.tsv', sep='\t')
# filter to S gene only
mutations = mutations[[m[:2]=='S:' for m in mutations.mutation]]
print("Our model considers {} mutations in the S gene.".format(len(mutations)))
# filter to sites that we have escape data for
mutations = mutations[[int(m[3:-1]) in sites for m in mutations.mutation]]
sites = list(set([int(m[3:-1]) if int(m[3:-1]) in sites else None for m in mutations.mutation]))
print("Of these, {} mutations have escape data:".format(len(sites)))
print(sorted(sites))

### All lineage analysis

In [ ]:
_, rbd_mutations, _, rbd_coef, _, _, rbd_features, clades, clade_to_lineage = \
    pickle.load(open('rbd_data.pkl', 'rb'))

print(len(rbd_mutations), rbd_coef.shape, rbd_features.shape)

common_mutations = []
for m in rbd_mutations:
    m = int(m[3:-1])
    if m in sites:
        common_mutations.append(1)
    else:
        common_mutations.append(0)
common_mutations = np.array(common_mutations, dtype=bool)   
print(common_mutations.sum(), common_mutations.shape[0])

In [ ]:
rbd_mutations = np.array(rbd_mutations)[common_mutations]
rbd_features = rbd_features[:, common_mutations]

at_least_one_rbd_mutation = rbd_features.sum(-1) > 0
rbd_features = np.array(rbd_features[at_least_one_rbd_mutation], dtype=bool)
print(rbd_features.shape)

rbd_pred = 0.01 * rbd_features @ rbd_coef[common_mutations]
print(rbd_pred.shape)

In [ ]:
binding = []

for variant in rbd_features:
    mutations = rbd_mutations[variant]
    mutations = [int(m[3:-1]) for m in mutations]
    assert len(mutations) == len(set(mutations))
    binding.append( 1 - bindcalc.binding_retained(mutations) )
        
print(len(binding))

In [ ]:
spearman = stats.spearmanr(binding, rbd_pred)[0]

plt.scatter(binding, rbd_pred, alpha=0.5, s=12)
plt.xlabel("1 - binding_retained (Greaney et. al.)", fontsize=16)
plt.ylabel("Δ log R (RBD only)", fontsize=16)
plt.text(0.0, 0.52, "$\\rho_{\\rm spearman}$" +": {:.2f}".format(spearman), fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.tick_params(axis='both', which='minor', labelsize=14)
    
plt.tight_layout()
plt.savefig("binding_retained.png")
plt.show()